# Muhammad Fozan
# 19I-0507

# Reading tweets from file
# Removing links

In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from nltk import word_tokenize, sent_tokenize
import string
df = pd.read_excel('realdonaldtrump.csv.xlsx')

# making list of tweets
tweets = df['content'].copy()
no_links = []

# Removing links

for row in tweets:
    clear_txt =re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', row, flags=re.MULTILINE)
    clear_txt = re.sub('pic.\S+', '', clear_txt)
    no_links.append(clear_txt)

print(no_links)




# Part 1
# 80% train 20% test

In [ ]:
train, testing = train_test_split(no_links, test_size=0.2, random_state=1, shuffle=False)
print(train)
# train = no_links

# Part2: Removing All punctuations expect '.'

In [ ]:
string.punctuation = string.punctuation +'“'+'”'+'-'+'’'+'‘'+'—'
string.punctuation = string.punctuation.replace('.', '')

content = []
for row in train:
  file_p = "".join([char for char in row if char not in string.punctuation])
  content.append(file_p)

print(content)



# Sentence tokenization

In [ ]:

token_sents = []
for row in content:
    sentence = sent_tokenize(row)
    if sentence != "":
        token_sents.extend(sentence)
token_sents


In [ ]:
# file_with_nl_removed = ""
# for line in content:
#   line_with_nl_removed = line.replace("\n", " ")
#   file_with_nl_removed += line_with_nl_removed
# file_with_nl_removed += "\n"
# print(file_with_nl_removed)

# Part 4: Removing stopwords

In [ ]:
from multiprocessing.resource_sharer import stop
stop_words = set(stopwords.words("english"))
filter_rows = []
for row in range(len(token_sents)):
    word = word_tokenize(token_sents[row])

    st_word = ""
    for j in word:
    # if curr word is not stopword then add it to filtered
        if j.casefold() not in stop_words:
            st_word+=' '
            st_word += j
    
    filter_rows.append(st_word)

# Again removing punctuations for . and -
string.punctuation = string.punctuation +'“'+'”'+'-'+'’'+'‘'+'—'+'.'+'–'

filter_text = []
for row in filter_rows:
  file_p = "".join([char for char in row if char not in string.punctuation])
  filter_text.append(file_p)

print(filter_text)


# Part 5: Building Bigrams

In [ ]:
from nltk.util import ngrams


bigrams_data = []

for row in filter_text:
    data = row.split(" ")
    bigrams = ngrams(data, 2)

    for i in bigrams:
        bigrams_data.append(i)

stops = [""]
new_bigrams = [gram for gram in bigrams_data if not any(stop in gram for stop in stops)]
# print(new_bigrams)
print(row)


# Part:6 Find Frequencies and make histogram

In [ ]:
# making frequency distribution of bigrams
bi_freq_dist = nltk.FreqDist(new_bigrams)
bi_freq_common = bi_freq_dist.most_common(40)
bi_freq_common

# Plotting Histogram

In [ ]:
bi_values = []
for tuple in bi_freq_common:
    bi_values.append(tuple[1])

from matplotlib import pyplot as plt
plt.hist(bi_values,10)


# Part:5 Making trigrams

In [ ]:
trigrams_data = []

for row in filter_text:
    data = row.split(" ")
    trigrams = ngrams(data, 3)

    for i in trigrams:
        trigrams_data.append(i)

# Removing trigrams involving empty strings
stops = [""]
new_trigrams = [gram for gram in trigrams_data if not any(stop in gram for stop in stops)]
print(new_trigrams)
# print(trigrams_data)

# Part:6 Make Frequency Dist and Histogram

In [ ]:
tri_freq_dist = nltk.FreqDist(new_trigrams)
tr_freq_common = tri_freq_dist.most_common(40)
tr_freq_common

In [ ]:
# Making histogram of most commin 40
tri_values = []
for tuple in tr_freq_common:
    tri_values.append(tuple[1])

# tri_values
from matplotlib import pyplot as plt
plt.hist(tri_values,10)

# Part:7 Laplace smoothing for bigrams and tri-grams

In [ ]:
for word in bi_freq_dist.keys():
    bi_freq_dist[word] += 1


for word in tri_freq_dist.keys():
    tri_freq_dist[word] += 1
tri_freq_dist.values()



# Part: 8 Build dictionaries for bigram and trigram and sort in increasing order
# For Bigrams

In [ ]:
bigram_probs = {"Word":[], "Probability":[]}
bigram_voc = set(bi_freq_dist)
total_freq = 0
for word in bi_freq_dist.keys():
    total_freq += bi_freq_dist[word]

for word in bi_freq_dist.keys():
    probability = (bi_freq_dist[word]/(total_freq + len(bigram_voc)))
    bigram_probs["Word"].append(word)
    bigram_probs["Probability"].append(probability)


# print(type(bigram_probs))
bigram_probs


In [ ]:
# Sorting the dictionary in increasing order 
import numpy as np


idx = np.argsort(bigram_probs["Probability"])
d = np.dtype(int)
bigram_probs["Probability"] = list(np.array(bigram_probs["Probability"])[idx])
dt=np.dtype('object,object')
bigram_probs["Word"] = list(np.array(bigram_probs["Word"],dtype=dt)[idx])
print(bigram_probs["Word"][len(bigram_probs["Word"])-1])
print(bigram_probs["Probability"][len(bigram_probs["Word"])-1])


# Part: 8 Build dictionaries for bigram and trigram and sort in increasing order
# For Trigrams

In [ ]:
trigram_probs = {"Word":[], "Probability":[]}
trigram_voc = set(tri_freq_dist)
total_freq = 0
for word in tri_freq_dist.keys():
    total_freq += tri_freq_dist[word]

for word in tri_freq_dist.keys():
    probability = (tri_freq_dist[word]/(total_freq + len(trigram_voc)))
    trigram_probs["Word"].append(word)
    trigram_probs["Probability"].append(probability)

trigram_probs

In [ ]:
# Sorting the dictionary in increasing order
import numpy as np


idx = np.argsort(trigram_probs["Probability"])
d = np.dtype(int)
trigram_probs["Probability"] = list(np.array(trigram_probs["Probability"])[idx])
dt=np.dtype('object,object,object')
trigram_probs["Word"] = list(np.array(trigram_probs["Word"],dtype=dt)[idx])
print(trigram_probs["Word"][len(trigram_probs["Word"])-1])
print(trigram_probs["Probability"][len(trigram_probs["Word"])-1])

# Part:10 Redo the histogram

In [ ]:
bi_freq_common = bi_freq_dist.most_common(40)
bi_values = []
for tuple in bi_freq_common:
    bi_values.append(tuple[1])

from matplotlib import pyplot as plt
plt.hist(bi_values,10)


# Part:10 Redo the Histogram

In [ ]:
tr_freq_common = tri_freq_dist.most_common(40)
tri_values = []
for tuple in tr_freq_common:
    tri_values.append(tuple[1])

# tri_values
from matplotlib import pyplot as plt
plt.hist(tri_values,10)

In [ ]:
# Building unigrams to be used in Part 11
unigrams=[]
for sent in filter_text:
    word_sequence=word_tokenize(sent)
    for word in word_sequence:
        if (word == '.'):
            word_sequence.remove(word)
        else:
            unigrams.append(word)

frequency_unigrams=nltk.FreqDist(unigrams)

# Part:11 Take a four-word string as the test sample and find both its unigram, bigram, and trigram perplexities

In [ ]:
str4="Did you know Donald"
testset=word_tokenize(str4)

#unigram
perplexity = 1
N = 0
for word in testset:
    if word in frequency_unigrams.keys():
        N += 1
        perplexity = perplexity * (1/frequency_unigrams[word])
if N == 0:
    perplexity = 1
else :
    perplexity = pow(perplexity, 1/float(N))

perplexity

In [ ]:
data= str4.split(" ")
test_bigrm=ngrams(data,2)

perplexity = 1
N = 0

list=[]
list.extend(test_bigrm)

for word in list:
    if word in bi_freq_dist.keys():
        N += 1
        perplexity = perplexity * (1/bi_freq_dist[word])
if N == 0:
    perplexity = 1
else :
    perplexity = pow(perplexity, 1/float(N))
perplexity

In [ ]:
data= str4.split(" ")
test_trigrm=ngrams(data,3)

perplexity = 1
N = 0

list=[]
list.extend(test_trigrm)


for word in list:
    if word in tri_freq_dist.keys():
        N += 1
        perplexity = perplexity * (1/tri_freq_dist[word])
if N == 0:
    perplexity = 1
else :
    perplexity = pow(perplexity, 1/float(N))
perplexity

# Part:12 Natural Langauge Generation Task: Based on step 7, build CDF (Cumulative Distribution Function). Generate a random number and compare it with the CDF. Pick the word whose CDF value is closest to it and return it as the generated text.

In [ ]:
import random
N=len(trigram_probs["Probability"])
data=trigram_probs["Probability"]
plt.xlabel('probability')
plt.ylabel('y-axis')
x=np.sort(data)
y=np.arange(N)/float(N)
plt.plot(x, y, marker='o',label='trigram')
num = random.uniform(0, max(x))
nearest=x[min(range(len(x)), key = lambda i: abs(x[i]-num))]
ind1=np.where(x==nearest)

N=len(bigram_probs["Probability"])
data=bigram_probs["Probability"]
x=np.sort(data)
y=np.arange(N)/float(N)

plt.plot(x, y, marker='x',label="'bigram'")
plt.title('CDF (Cumulative Distribution Function)')
num = random.uniform(0, max(x))
print("Random Number : ", num)
nearest=x[min(range(len(x)), key = lambda i: abs(x[i]-num))]
ind=np.where(x==nearest)
print("Closest Bigram : ",bigram_probs["Word"][ind[0][0]])
print("Closest Trigram : ",trigram_probs["Word"][ind1[0][0]])

# Part:13 Shannon Game

In [ ]:
string.punctuation = string.punctuation +'“'+'”'+'-'+'’'+'‘'+'—' + '.' + '…'

test_content = []
for row in testing:
  file_p = "".join([char for char in row if char not in string.punctuation])
  test_content.append(file_p)

test_tokens = []
for row in test_content:
    sentence = sent_tokenize(row)
    if sentence != "":
        test_tokens.extend(sentence)
# Preparing Test Y for comparison
test_Y = []
for sent in test_tokens:
  last_word = ""
  for ch in range(len(sent)):
    
    if sent[ch] == ' ':
      last_word = ""
    else:
      last_word += sent[ch]
  test_Y.append(last_word)








In [ ]:
# 2nd last words
sec_lasts = []
for sent in test_tokens:
  last2 = ""
  counter = len(sent)-1
  start = 0
  while counter >= 0:
    if start and sent[counter]==" ":
      break
    elif start:
      last2 += sent[counter]
    if(sent[counter]==" "):
      start = 1
    

    counter -= 1
  sec_lasts.append(last2[::-1])
print(sec_lasts)

In [ ]:
bi_output = []
all_keys = bi_freq_dist.keys()
all_values = bi_freq_dist.values()

print(all_keys)
# for word in sec_lasts:
#     # find all bigrams
#     freq = 0
#     if word == "":
#         bi_output.append("UNKNOWN")
#     else:
#         for i in range(len(all_keys)):
#             j = i
#             # if all_keys[i][0] == word:
#     #             if all_values[i] > freq:
#     #                 bi_predict = all_keys[i][0]
#     #                 freq = all_values[i]
#         # if freq == 0:
#         #     bi_output.append("UNKNOWN")
#         # else:
#         #     bi_output.append(bi_predict)
# print(bi_output)

